In [1]:
from absenteeism_module import *

In [31]:
pd.read_csv('Absenteeism_data.csv')

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,7/7/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
5,3,23,10/7/2015,179,51,38,239.554,31,1,0,0,2
6,10,22,17/07/2015,361,52,28,239.554,27,1,1,4,8
7,20,23,24/07/2015,260,50,36,239.554,23,1,4,0,4
8,14,19,6/7/2015,155,12,34,239.554,25,1,2,0,40
9,1,22,13/07/2015,235,11,37,239.554,29,3,1,1,8


In [32]:
model = absenteeism_model('model', 'scaler')

C:\Users\Son\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Son\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Input raw data into the model to clean it

In [33]:
model.load_and_clean_data('Absenteeism_data.csv')

Now let's input clean data to the model to predict.

**Note**: Using this ML model to pre-porcess the data will drop additionally the 'Daily Work Load Average' column

In [34]:
predicted_data = model.predicted_outputs()

Note this is not exactly the name of the table I created in mySQL. Because later after I established the connection between Jupyter and Python I will create a checkpoint in case something went wrong with the data. The copy variable of the 'predicted_data' will be the name of the table I created in mySQL

In [35]:
predicted_data

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pet,Probability,Prediction
0,0,0,0,1,7,289,33,30,0,2,1,0.593426,1
1,0,0,0,0,7,118,50,31,0,1,0,0.087617,0
2,0,0,0,1,7,179,38,31,0,0,0,0.245968,0
3,1,0,0,0,7,279,39,24,0,2,0,0.871887,1
4,0,0,0,1,7,289,33,30,0,2,1,0.593426,1
5,0,0,0,1,7,179,38,31,0,0,0,0.245968,0
6,0,0,0,1,7,361,28,27,0,1,4,0.491962,0
7,0,0,0,1,7,260,36,23,0,4,0,0.604556,1
8,0,0,1,0,7,155,34,25,0,2,0,0.786172,1
9,0,0,0,1,7,235,37,29,1,1,1,0.292227,0


The 'Probability' column is about a given individual is expected to be absent from work for more than 3 hours (excessively)

The 'Prediction' column:

            1 -  the probability obtained was 50% or higher
            
            0 -  the probability obtained was lower than 50%

In [36]:
import pymysql

Let's create a connection to mySQL:

In [37]:
connect = pymysql.connect(database = 'predicted_data', user = 'root', password = 'Legendary95')

Now I can write codes in Jupyter as you're writting codes in mySQL. But I need to a cursor to know where I can to write or execute my codes

In [38]:
cursor = connect.cursor()

Now I will choose the table I've created in mySQL to start inserting data in

In [39]:
cursor.execute('SELECT * FROM pre_data_c;')

0

Let's start sending data from Jupyter to mySQL:

In [40]:
insert_query = 'INSERT INTO pre_data_c VALUES '

According to the SQL syntax to insert data in the table:
                                               
                                               INSERT INTO table_names VALUES 
                                               (value, value),
                                               (value, value),
                                               ... ;
                                               
I will add **space** after **VALUES**, and the rest I will add using the loop syntax 

In [41]:
predicted_data.shape

(700, 13)

Below means extracting all columns' values, put all of those values into a list. I just want to see how many columns I have

In [42]:
predicted_data.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pet', 'Probability', 'Prediction'], dtype=object)

Let's create a copy in case something went wrong with the data:

In [43]:
pre_data_c = predicted_data.copy()

As I mentioned below cell 5 this is the name I named my insert-data table in mySQL. 

In [44]:
pre_data_c

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pet,Probability,Prediction
0,0,0,0,1,7,289,33,30,0,2,1,0.593426,1
1,0,0,0,0,7,118,50,31,0,1,0,0.087617,0
2,0,0,0,1,7,179,38,31,0,0,0,0.245968,0
3,1,0,0,0,7,279,39,24,0,2,0,0.871887,1
4,0,0,0,1,7,289,33,30,0,2,1,0.593426,1
5,0,0,0,1,7,179,38,31,0,0,0,0.245968,0
6,0,0,0,1,7,361,28,27,0,1,4,0.491962,0
7,0,0,0,1,7,260,36,23,0,4,0,0.604556,1
8,0,0,1,0,7,155,34,25,0,2,0,0.786172,1
9,0,0,0,1,7,235,37,29,1,1,1,0.292227,0


In [45]:
for i in range(pre_data_c.shape[0]):
    insert_query += '('
    
    for j in range(pre_data_c.shape[1]):
        insert_query += str(pre_data_c[pre_data_c.columns.values[j]][i]) + ', '
        
    insert_query = insert_query[:-2] + '), '

let's check for my query:

In [46]:
insert_query

'INSERT INTO pre_data_c VALUES (0, 0, 0, 1, 7, 289, 33, 30, 0, 2, 1, 0.5934258852199636, 1), (0, 0, 0, 0, 7, 118, 50, 31, 0, 1, 0, 0.08761650041469149, 0), (0, 0, 0, 1, 7, 179, 38, 31, 0, 0, 0, 0.24596823965505082, 0), (1, 0, 0, 0, 7, 279, 39, 24, 0, 2, 0, 0.8718871597374543, 1), (0, 0, 0, 1, 7, 289, 33, 30, 0, 2, 1, 0.5934258852199636, 1), (0, 0, 0, 1, 7, 179, 38, 31, 0, 0, 0, 0.24596823965505082, 0), (0, 0, 0, 1, 7, 361, 28, 27, 0, 1, 4, 0.4919621401501115, 0), (0, 0, 0, 1, 7, 260, 36, 23, 0, 4, 0, 0.604555761272743, 1), (0, 0, 1, 0, 7, 155, 34, 25, 0, 2, 0, 0.7861722459844033, 1), (0, 0, 0, 1, 7, 235, 37, 29, 1, 1, 1, 0.2922274469749605, 0), (1, 0, 0, 0, 7, 260, 36, 23, 0, 4, 0, 0.9160325023634209, 1), (1, 0, 0, 0, 7, 260, 36, 23, 0, 4, 0, 0.9160325023634209, 1), (1, 0, 0, 0, 7, 260, 36, 23, 0, 4, 0, 0.9160325023634209, 1), (1, 0, 0, 0, 7, 179, 38, 31, 0, 0, 0, 0.6994974408568505, 1), (0, 0, 0, 1, 7, 179, 38, 31, 0, 0, 0, 0.24596823965505082, 0), (1, 0, 0, 0, 7, 246, 41, 23, 0, 0, 0

Look at the end, the out doesn't end with ' ; ' a proper statement terminator. Let's use reindexing to fix this:

In [47]:
insert_query = insert_query[:-2] + ';'

In [48]:
insert_query

'INSERT INTO pre_data_c VALUES (0, 0, 0, 1, 7, 289, 33, 30, 0, 2, 1, 0.5934258852199636, 1), (0, 0, 0, 0, 7, 118, 50, 31, 0, 1, 0, 0.08761650041469149, 0), (0, 0, 0, 1, 7, 179, 38, 31, 0, 0, 0, 0.24596823965505082, 0), (1, 0, 0, 0, 7, 279, 39, 24, 0, 2, 0, 0.8718871597374543, 1), (0, 0, 0, 1, 7, 289, 33, 30, 0, 2, 1, 0.5934258852199636, 1), (0, 0, 0, 1, 7, 179, 38, 31, 0, 0, 0, 0.24596823965505082, 0), (0, 0, 0, 1, 7, 361, 28, 27, 0, 1, 4, 0.4919621401501115, 0), (0, 0, 0, 1, 7, 260, 36, 23, 0, 4, 0, 0.604555761272743, 1), (0, 0, 1, 0, 7, 155, 34, 25, 0, 2, 0, 0.7861722459844033, 1), (0, 0, 0, 1, 7, 235, 37, 29, 1, 1, 1, 0.2922274469749605, 0), (1, 0, 0, 0, 7, 260, 36, 23, 0, 4, 0, 0.9160325023634209, 1), (1, 0, 0, 0, 7, 260, 36, 23, 0, 4, 0, 0.9160325023634209, 1), (1, 0, 0, 0, 7, 260, 36, 23, 0, 4, 0, 0.9160325023634209, 1), (1, 0, 0, 0, 7, 179, 38, 31, 0, 0, 0, 0.6994974408568505, 1), (0, 0, 0, 1, 7, 179, 38, 31, 0, 0, 0, 0.24596823965505082, 0), (1, 0, 0, 0, 7, 246, 41, 23, 0, 0, 0

Now I have the string we can deliver to mySQL

In [49]:
cursor.execute(insert_query)

700

When you refresh SQL you'll see that the table is empty. It's because you've executed the statement but you haven't commited it. Let's commit it

In [50]:
connect.commit()

Now I have finished my work so I will close the connection between Jupyter and SQL

In [51]:
connect.close()